In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

Chain of Density: https://arxiv.org/pdf/2309.04269.pdf


In [11]:
import time
import textwrap

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.schema.runnable import RunnablePassthrough

# Load some data to summarize
loader = WebBaseLoader("https://teddylee777.github.io/data-science/optuna/")
docs = loader.load()
content = docs[0].page_content

# Get this prompt template
prompt = hub.pull("lawwu/chain_of_density")

# The chat model output is a JSON list of dicts, with SimpleJsonOutputParser
# we can convert it o a dict, and it suppors streaming.
json_parser = SimpleJsonOutputParser()

In [16]:
chain = (
    {"ARTICLE": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.1)
    | json_parser
)

In [17]:
chain.invoke(content)

[{'Missing_Entities': 'Optuna; data science',
  'Denser_Summary': 'This article discusses Optuna, a data science tool. Optuna is a library for hyperparameter optimization. It provides a simple and efficient way to search for the best hyperparameters for machine learning models. With Optuna, you can easily tune the parameters of your models and improve their performance. It is a powerful tool for data scientists and machine learning practitioners.'},
 {'Missing_Entities': 'trial.suggest_categorical(); trial.suggest_int(); trial.suggest_float()',
  'Denser_Summary': 'Optuna provides several methods for suggesting hyperparameters. One of them is trial.suggest_categorical(), which allows you to choose from a set of predefined values. Another method is trial.suggest_int(), which suggests integer values within a specified range. Lastly, trial.suggest_float() suggests floating-point values within a given range. These methods are useful for exploring different hyperparameter values and finding

In [101]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
import json


# Load some data to summarize
loader = WebBaseLoader(
    "https://www.aitimes.com/news/articleView.html?idxno=131777")
docs = loader.load()
content = docs[0].page_content
# Load the prompt
# prompt = hub.pull("langchain-ai/chain-of-density:4f55305e")


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


prompt = ChatPromptTemplate.from_template(
    """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (50 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""
)


# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-4-turbo-preview",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

```json
[
    {
        "Missing_Entities": "",
        "Denser_Summary": "이 기사는 데이터사이언스, 머신러닝, 인공지능에 대한 개념을 설명하고 있습니다. 이 분야들이 우리 생활에 어떻게 적용되고 있는지, 그리고 이 분야를 공부하고자 하는 사람들이 어떻게 시작해야 할지에 대한 정보를 제공합니다. 또한, 이 기술들이 어떻게 빠르게 발전하고 있는지에 대한 설명도 포함하고 있습니다. 이러한 기술들의 정의와 차이점, 그리고 이 분야에서 일하려면 어떤 준비가 필요한지에 대한 내용도 다루고 있습니다."
    },
    {
        "Missing_Entities": "박정현 서울대 EPM 연구원; '인공지능 국가전략' 발표; 퍼듀 대학의 윌리엄 클리블랜드 교수",
        "Denser_Summary": "박정현 서울대 EPM 연구원이 작성한 이 기사는 데이터사이언스, 머신러닝, 인공지능의 개념과 실생활 적용 예를 설명합니다. '인공지능 국가전략' 발표와 같은 국가적 노력과 퍼듀 대학의 윌리엄 클리블랜드 교수의 이론을 포함해, 이 분야의 학문적, 실용적 측면을 다룹니다. 또한, 이 기술들의 발전 속도와 이 분야에 대한 관심이 증가하는 이유를 설명하며, 이 분야를 공부하려는 이들에게 시작점을 제시합니다."
    },
    {
        "Missing_Entities": "튜링테스트; 지도학습(supervised learning)과 비지도 학습(unsupervised learning)",
        "Denser_Summary": "박정현 서울대 EPM 연구원이 작성한 이 기사는 데이터사이언스, 머신러닝, 인공지능의 정의와 '인공지능 국가전략', 퍼듀 대학의 윌리엄 클리블랜드 교수의 이론을 포함해 설명합니다. 튜링테스트와 지도학습, 비지도 학습의 개념을 통해 인공지능의 실용적 적용을 다루며, 이 분야의 발전과 공부 방향에 대한 조언을 제공합니다."
    },
    {
        "M